
# Data Engineering Capstone Project

#### Project Summary


The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import os
import configparser
import datetime as dt
import psycopg2

from pyspark.sql import SparkSession
from pyspark.sql.functions import avg
from pyspark.sql import SQLContext
from pyspark.sql.functions import isnan, when, count, col, udf, dayofmonth, dayofweek, month, year, weekofyear
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.types import *

import requests
requests.packages.urllib3.disable_warnings()

import utility
import functions

import importlib
importlib.reload(utility)
from utility import clean_spark_immigration_data, clean_spark_temperature_data
from utility import clean_spark_demographics_data, print_formatted_float

In [2]:
config = configparser.ConfigParser()
config.read('dl.cfg')

os.environ['AWS_ACCESS_KEY_ID']=config['KEYS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['KEYS']['AWS_SECRET_ACCESS_KEY']

In [3]:
spark = SparkSession.builder.\
    config("spark.jars.repositories", "https://repos.spark-packages.org/").\
    config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11").\
    enableHiveSupport().getOrCreate()



## Step 1: Scope the Project and Gather Data

#### Scope 
The objective of this project is to make analysis on the immigration events based on various datasets obtained. An ETL is to be created for I-94 immigration, global land temperature and US demographics datasets. This is used to get an idea of immigration patterns to US.

For example, we could try to find answears to questions such as, do people from countries with warmer or cold climate immigrate to the US in large numbers?
Or , Around which states are immigrants more drawn towards ? And so on..
#### Immigration Dataset
This data comes from the US National Tourism and Trade Office.In the past all foreign visitors to the U.S. arriving via air or sea were required to complete paper Customs and Border Protection Form I-94 Arrival/Departure Record or Form I-94W Nonimmigrant Visa Waiver Arrival/Departure Record and this dataset comes from this forms. This dataset forms the core of the data warehouse and the customer repository has a years worth of data for the year 2016 and the dataset is divided by month. For this project the data is in a folder located at ../../data/18-83510-I94-Data-2016/. Each months data is stored in an SAS binary database storage format sas7bdat. For this project we have chosen going to work with data for the month of April. The data extraction, transformation and loading utility functions have been designed to work with any month's worth of data.

In [4]:
# Read in the data here
immig_name = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
immigration_df =spark.read.format('com.github.saurfang.sas.spark').load(immig_name)



In [5]:
immigration_df.head()

Row(cicid=6.0, i94yr=2016.0, i94mon=4.0, i94cit=692.0, i94res=692.0, i94port='XXX', arrdate=20573.0, i94mode=None, i94addr=None, depdate=None, i94bir=37.0, i94visa=2.0, count=1.0, dtadfile=None, visapost=None, occup=None, entdepa='T', entdepd=None, entdepu='U', matflag=None, biryear=1979.0, dtaddto='10282016', gender=None, insnum=None, airline=None, admnum=1897628485.0, fltno=None, visatype='B2')

In [6]:
print_formatted_float(immigration_df.count())

3,096,313


In [7]:
immigration_df.select("visapost").dropDuplicates().show(5)

+--------+
|visapost|
+--------+
|     CRS|
|     KGL|
|     AKD|
|     BGM|
|     TRK|
+--------+
only showing top 5 rows



#### World Temperature Data
This dataset came from Kaggle accessible through '../../data2/GlobalLandTemperaturesByCity.csv' . The dataset provides data about global land temperatures by cities

In [8]:
file_name = '../../data2/GlobalLandTemperaturesByCity.csv'
temperature_df = spark.read.csv(file_name, header=True, inferSchema=True)
# display the first five records
temperature_df.limit(5).toPandas()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [9]:
# check the total number of records
print_formatted_float(temperature_df.count())

8,599,212


#### US Demographic Data
This data comes from OpenSoft. It contains information about the demographics of all US cities and census-designated places with a population greater or equal to 65,000. The original data source is the US Census Bureau's 2015 American Community Survey.

In [10]:
file_name = "us-cities-demographics.csv"
demographics_df = spark.read.csv(file_name, inferSchema=True, header=True, sep=';')
# display the first five records
demographics_df.limit(5).toPandas()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601,41862,82463,1562,30908,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129,49500,93629,4147,32935,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040,46799,84839,4819,8229,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127,87105,175232,5821,33878,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040,143873,281913,5829,86253,2.73,NJ,White,76402


In [11]:
# check the total number of records
print_formatted_float(demographics_df.count())

2,891


In [12]:
# from pyspark.sql import SparkSession

# spark = SparkSession.builder.\
# config("spark.jars.repositories", "https://repos.spark-packages.org/").\
# config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
# enableHiveSupport().getOrCreate()

# df_spark = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')


In [13]:
# #write to parquet
# df_spark.write.parquet("sas_data")
# df_spark=spark.read.parquet("sas_data")

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

### 1.Immigration dataset

In [14]:
# Performing cleaning tasks here

files = os.listdir('../../data/18-83510-I94-Data-2016/')
files


['i94_apr16_sub.sas7bdat',
 'i94_sep16_sub.sas7bdat',
 'i94_nov16_sub.sas7bdat',
 'i94_mar16_sub.sas7bdat',
 'i94_jun16_sub.sas7bdat',
 'i94_aug16_sub.sas7bdat',
 'i94_may16_sub.sas7bdat',
 'i94_jan16_sub.sas7bdat',
 'i94_oct16_sub.sas7bdat',
 'i94_jul16_sub.sas7bdat',
 'i94_feb16_sub.sas7bdat',
 'i94_dec16_sub.sas7bdat']

In [15]:
immigration_df.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

#### Data Dictionary - Immigaration Dataset

In [16]:
imm_dict =pd.read_csv('Data dictionaries/immigration-data-dictionary.csv')
imm_dict

,Feature,Description
0,cicid,Unique record ID
1,i94yr,4 digit year
2,i94mon,Numeric month
3,i94cit,3 digit code for immigrant country of birth
4,i94res,3 digit code for immigrant country of residence
5,i94port,Port of admission
6,arrdate,Arrival Date in the USA
7,i94mode,Mode of transportation (1 = Air; 2 = Sea; 3 = ...
8,i94addr,USA State of arrival
9,depdate,Departure Date from the USA


In [17]:
# columns with large amount of missing values as discovered by the exploratory analysis 
cols = ['occup', 'entdepu','insnum']

# drop these columns
new_immig_df = immigration_df.drop(*cols)

In [18]:
# display the new schema
new_immig_df.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = true)
 |-- fltno: string (nullable = true)
 |-- visatype: string (nullable = true)



In [19]:
# drop duplicate entries
new_immig_df = new_immig_df.dropDuplicates(['cicid'])

In [20]:
# get a count after dropping duplicates
print_formatted_float(new_immig_df.count())

3,096,313


In [21]:
# drop rows with missing values
new_immig_df = new_immig_df.dropna(how='all', subset=['cicid'])

In [22]:
# get a count after dropping rows with missing values
print_formatted_float(new_immig_df.count())

3,096,313


In [23]:
# clean the immigration dataframe
new_immigration_df = utility.clean_spark_immigration_data(immigration_df)

Total records in dataframe: 3,096,313
Total records after cleaning: 3,096,313


### 2.Temperature Dataset

In [24]:
# print temperature dataframe schema
temperature_df.printSchema()

root
 |-- dt: timestamp (nullable = true)
 |-- AverageTemperature: double (nullable = true)
 |-- AverageTemperatureUncertainty: double (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)



#### Data Dictionary - Temperature Dataset

In [25]:
temp_dict =pd.read_csv('Data dictionaries/Temperature-data-dictionary.csv')

In [26]:
temp_dict

,Feature,Description
0,dt,Date
1,AverageTemperature,Global average land temperature in celsius
2,AverageTemperatureUncertainty,95% confidence interval around the average
3,City,Name of City
4,Country,Name of Country
5,Latitude,City Latitude
6,Longitude,City Longitude


*Exploratory analysis showed some missing values of average temperature.Hence clean global temperature data by dropping the rows that has missing average temperature values and dropping duplicate columns.*

In [27]:
# clean the data by dropping rows with missing average temperature values and dropping duplicates
new_temperature_df = utility.clean_spark_temperature_data(temperature_df)

Total records in dataframe: 8,599,212
Total records after dropping rows with missing values: 364,130
Rows dropped after accounting for duplicates: 44,299


In [28]:
# count the number of records in dataset
print_formatted_float(demographics_df.count())

2,891


### 3.Demographics Dataset

In [29]:
# print demographics database schema
demographics_df.printSchema()

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Median Age: double (nullable = true)
 |-- Male Population: integer (nullable = true)
 |-- Female Population: integer (nullable = true)
 |-- Total Population: integer (nullable = true)
 |-- Number of Veterans: integer (nullable = true)
 |-- Foreign-born: integer (nullable = true)
 |-- Average Household Size: double (nullable = true)
 |-- State Code: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Count: integer (nullable = true)



#### Data Dictionary - Demographics Dataset

In [30]:
demo_dict =pd.read_csv('Data dictionaries/Demographic data dictionary.csv')
demo_dict

,Feature,Description
0,City,City Name
1,State,US State where city is located
2,Median Age,Median age of the population
3,Male Population,Count of male population
4,Female Population,Count of female population
5,Total Population,Count of total population
6,Number of Veterans,Count of total Veterans
7,Foreign born,Count of residents of the city that were not b...
8,Average Household Size,Average city household size
9,State Code,Code of the US state


*Exploratory analysis showed very minimal missing values in the demographics dataset.Hence clean demographics data by dropping the rows that contain missing values and dropping duplicate columns.*

In [31]:
# clean demographics data by dropping the rows that contain missing values and dropping duplicate columns
new_demographics_df = utility.clean_spark_demographics_data(demographics_df)

Rows dropped with missing values: 16
Rows dropped after accounting for duplicates: 0


### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model



### Database schema
#### Fact Table
- The immigration fact table is the fact table of the data model. This table's data comes from the immigration data sets and contains keys that links to the dimension tables. 

#### Dimension Tables
- The *country* dimension table is made up of data from the global land temperatures by city and the immigration datasets. The combination of these two  datasets allows analysts to study correlations between global land temperatures and immigration patterns to the US.

- The *us_demographics* dimension table comes from the demographics dataset and links to the immigration fact table at US state level. This dimension would allow analysts to get insights into migration patterns into the US based on demographics as well as overall population of states.It gives an insight on which states most immigrants are drawn towards

- The *immigration_calendar* dimension table formed from the immigration datasets points towards arrival time of the immigrants

- The *visa_type* dimension table comes from the immigration datasets and links to the immigaration making use of the visa_type_key.

#### 3.2 Mapping Out Data Pipelines
 **The pipeline steps are as follows:**

- Run functions.py and utility.py in the console to enable the functions for cleaning the datasets and creating the tables
- Load the datasets
- Cleaning the I94 Immigration data to create Spark dataframe for each month
- Create visa_type dimension table
- Create calendar dimension table
- Extract clean global temperatures data
- Create country dimension table
- Create immigration fact table
- Load demographics data
- Clean demographics data
- Create demographic dimension table

In [32]:
# import image module
from IPython.display import Image
  
# get the image
Image(url="Data model.png", width=700, height=700)

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

#### I. Create the immigration calendar dimension table

In [33]:
# Write code here

def create_immigration_calendar_dimension(df, output_data):
    """This function creates an immigration calendar based on arrival date
    
    :param df: spark dataframe of immigration events
    :param output_data: path to write dimension dataframe to
    :return: spark dataframe representing calendar dimension
    """
    # create a udf to convert arrival date in SAS format to datetime object
    get_datetime = udf(lambda x: (dt.datetime(1960, 1, 1).date() + dt.timedelta(x)).isoformat() if x else None)
    
    # create initial calendar df from arrdate column
    calendar_df = df.select(['arrdate']).withColumn("arrdate", get_datetime(df.arrdate)).distinct()
    
    # expand df by adding other calendar columns
    calendar_df = calendar_df.withColumn('arrival_day', dayofmonth('arrdate'))
    calendar_df = calendar_df.withColumn('arrival_week', weekofyear('arrdate'))
    calendar_df = calendar_df.withColumn('arrival_month', month('arrdate'))
    calendar_df = calendar_df.withColumn('arrival_year', year('arrdate'))
    calendar_df = calendar_df.withColumn('arrival_weekday', dayofweek('arrdate'))

    # create an id field in calendar df
    calendar_df = calendar_df.withColumn('id', monotonically_increasing_id())
    
    # write the calendar dimension to parquet file
    partition_columns = ['arrival_year', 'arrival_month', 'arrival_week']
    calendar_df.write.parquet(output_data + "immigration_calendar", partitionBy=partition_columns, mode="overwrite")
    
    return calendar_df

In [34]:
output_data = "tables/"

In [35]:
calendar_df = create_immigration_calendar_dimension(new_immigration_df, output_data)

In [36]:
calendar_df.limit(5).toPandas()

,arrdate,arrival_day,arrival_week,arrival_month,arrival_year,arrival_weekday,id
0,2016-04-22,22,16,4,2016,6,8589934592
1,2016-04-15,15,15,4,2016,6,25769803776
2,2016-04-18,18,16,4,2016,2,42949672960
3,2016-04-09,9,14,4,2016,7,68719476736
4,2016-04-11,11,15,4,2016,2,85899345920


#### II. Create the country dimension table

In [37]:
def create_country_dimension_table(df, temp_df, output_data):
    """This function creates a country dimension from the immigration and global land temperatures data.
    
    :param df: spark dataframe of immigration events
    :temp_df: spark dataframe of global land temperatures data.
    :param output_data: path to write dimension dataframe to
    :return: spark dataframe representing calendar dimension
    """
    # get the aggregated temperature data
    agg_temp = utility.aggregate_temperature_data(temp_df).toPandas()
    # load the i94res to country mapping data
    mapping_codes = pd.read_csv('i94.csv')
    
    @udf('string')
    def get_country_average_temperature(name):
        print("Processing: ", name)
        avg_temp = agg_temp[agg_temp['Country']==name]['average_temperature']
        
        if not avg_temp.empty:
            return str(avg_temp.iloc[0])
        
        return None
    
    @udf()
    def get_country_name(code):
        name = mapping_codes[mapping_codes['code']==code]['name'].iloc[0]
        
        if name:
            return name.title()
        return None
        
    # select and rename i94res column
    dim_df = df.select(['i94res']).distinct() \
                .withColumnRenamed('i94res', 'country_code')
    
    # create country_name column
    dim_df = dim_df.withColumn('country_name', get_country_name(dim_df.country_code))
    
    # create average_temperature column
    dim_df = dim_df.withColumn('average_temperature', get_country_average_temperature(dim_df.country_name))
    
    # write the dimension to a parquet file
    dim_df.write.parquet(output_data + "country", mode="overwrite")
    
    return dim_df

In [38]:
country_dim_df = create_country_dimension_table(new_immigration_df, new_temperature_df, output_data)


In [39]:
country_dim_df.show(5)

+------------+-------------+-------------------+
|country_code| country_name|average_temperature|
+------------+-------------+-------------------+
|       692.0|      Ecuador|               null|
|       299.0|     Mongolia|               null|
|       576.0|  El Salvador|               null|
|       735.0|  Montenegro |               null|
|       206.0|    Hong Kong|               null|
+------------+-------------+-------------------+
only showing top 5 rows



#### III. Create the visa_type dimension table

In [40]:
def create_visa_type_dimension_table(df, output_data):
    """This function creates a visa type dimension from the immigration data.
    
    :param df: spark dataframe of immigration events
    :param output_data: path to write dimension dataframe to
    :return: spark dataframe representing calendar dimension
    """
    # create visatype df from visatype column
    visatype_df = df.select(['visatype']).distinct()
    
    # add an id column
    visatype_df = visatype_df.withColumn('visa_type_key', monotonically_increasing_id())
    
    # write dimension to parquet file
    visatype_df.write.parquet(output_data + "visatype", mode="overwrite")
    
    return visatype_df

def get_visa_type_dimension(output_data):
    return spark.read.parquet(output_data + "visatype")

In [41]:
visatype_df = create_visa_type_dimension_table(new_immigration_df, output_data)
visatype_df.show(n=5)

+--------+-------------+
|visatype|visa_type_key|
+--------+-------------+
|      F2| 103079215104|
|     GMB| 352187318272|
|      B2| 369367187456|
|      F1| 498216206336|
|     CPL| 601295421440|
+--------+-------------+
only showing top 5 rows



#### IV. Create the demographics dimension table

In [42]:
def create_demographics_dimension_table(df, output_data):
    """This function creates a us demographics dimension table from the us cities demographics data.
    
    :param df: spark dataframe of us demographics survey data
    :param output_data: path to write dimension dataframe to
    :return: spark dataframe representing demographics dimension
    """
    dim_df = df.withColumnRenamed('Median Age','median_age') \
            .withColumnRenamed('Male Population', 'male_population') \
            .withColumnRenamed('Female Population', 'female_population') \
            .withColumnRenamed('Total Population', 'total_population') \
            .withColumnRenamed('Number of Veterans', 'number_of_veterans') \
            .withColumnRenamed('Foreign-born', 'foreign_born') \
            .withColumnRenamed('Average Household Size', 'average_household_size') \
            .withColumnRenamed('State Code', 'state_code')
    # lets add an id column
    dim_df = dim_df.withColumn('id', monotonically_increasing_id())
    
    # write dimension to parquet file
    dim_df.write.parquet(output_data + "demographics", mode="overwrite")
    
    return dim_df

In [43]:
demographics_dim_df = create_demographics_dimension_table(demographics_df, output_data)
demographics_dim_df.limit(5).toPandas()

,City,State,median_age,male_population,female_population,total_population,number_of_veterans,foreign_born,average_household_size,state_code,Race,Count,id
0,Silver Spring,Maryland,33.8,40601,41862,82463,1562,30908,2.60,MD,Hispanic or Latino,25924,0
1,Quincy,Massachusetts,41.0,44129,49500,93629,4147,32935,2.39,MA,White,58723,1
2,Hoover,Alabama,38.5,38040,46799,84839,4819,8229,2.58,AL,Asian,4759,2
3,Rancho Cucamonga,California,34.5,88127,87105,175232,5821,33878,3.18,CA,Black or African-American,24437,3
4,Newark,New Jersey,34.6,138040,143873,281913,5829,86253,2.73,NJ,White,76402,4


#### V. Create the immigration Fact table

In [44]:
def create_immigration_fact_table(df, output_data):
    """This function creates an country dimension from the immigration and global land temperatures data.
    
    :param df: spark dataframe of immigration events
    :param visa_type_df: spark dataframe of global land temperatures data.
    :param output_data: path to write dimension dataframe to
    :return: spark dataframe representing calendar dimension
    """
    # get visa_type dimension
    dim_df = get_visa_type_dimension(output_data).toPandas()
    
    @udf('string')
    def get_visa_key(visa_type):
        """user defined function to get visa key
        
        :param visa_type: US non-immigrant visa type
        :return: corresponding visa key
        """
        key_series = dim_df[dim_df['visatype']==visa_type]['visa_type_key']
        
        if not key_series.empty:
            return str(key_series.iloc[0])
        
        return None
    
    # create a udf to convert arrival date in SAS format to datetime object
    get_datetime = udf(lambda x: (dt.datetime(1960, 1, 1).date() + dt.timedelta(x)).isoformat() if x else None)
    
    # rename columns to align with data model
    df = df.withColumnRenamed('cicid','record_id') \
            .withColumnRenamed('i94res', 'country_residence_code') \
            .withColumnRenamed('i94addr', 'state_code') 
    
    # create visa_type key
    df = df.withColumn('visa_type_key', get_visa_key('visatype'))
    
    # convert arrival date into datetime object
    df = df.withColumn("arrdate", get_datetime(df.arrdate))
    
    # write dimension to parquet file
    df.write.parquet(output_data + "immigration_fact", mode="overwrite")
    
    return df

In [45]:
immigration_fact_df = create_immigration_fact_table(new_immigration_df, output_data)

In [46]:
immigration_fact_df.limit(5).toPandas()

,record_id,i94yr,i94mon,i94cit,country_residence_code,i94port,arrdate,i94mode,state_code,depdate,...,entdepd,matflag,biryear,dtaddto,gender,airline,admnum,fltno,visatype,visa_type_key
0,299.0,2016.0,4.0,103.0,103.0,NYC,2016-04-01,1.0,NY,20550.0,...,O,M,1962.0,06292016,None,OS,5.542587e+10,00087,WT,884763262976
1,305.0,2016.0,4.0,103.0,103.0,NYC,2016-04-01,1.0,NY,20555.0,...,O,M,1953.0,06292016,None,OS,5.542582e+10,00087,WT,884763262976
2,496.0,2016.0,4.0,103.0,103.0,CHI,2016-04-01,1.0,IL,20548.0,...,O,M,1952.0,06292016,None,OS,5.542862e+10,00065,WB,738734374912
3,558.0,2016.0,4.0,103.0,103.0,SFR,2016-04-01,1.0,CA,20547.0,...,O,M,1974.0,06292016,M,LH,5.543331e+10,00454,WB,738734374912
4,596.0,2016.0,4.0,103.0,103.0,NAS,2016-04-01,1.0,FL,20547.0,...,N,M,1992.0,06292016,M,UP,5.540611e+10,00221,WT,884763262976


In [47]:
immigration_fact_df.createOrReplaceTempView('immigration_fact_table')
visatype_df.createOrReplaceTempView('visatype_table')
demographics_dim_df.createOrReplaceTempView('demographics_dim_table')
country_dim_df.createOrReplaceTempView('country_dim_table')
calendar_df.createOrReplaceTempView('calendar_table')

**Query on count of people of different visatypes living in a particular state**

In [48]:
query= spark.sql('''SELECT vt.visatype,dg.state,count(*)
     FROM immigration_fact_table if
     
     JOIN  visatype_table vt
     ON vt.visa_type_key = if.visa_type_key
     JOIN demographics_dim_table dg
     ON dg.state_code =if.state_code
     GROUP BY 1,2
     ORDER by 3 DESC
     ''')



In [49]:
query.show()

+--------+----------+---------+
|visatype|     state| count(1)|
+--------+----------+---------+
|      WT|California|132392572|
|      B2|California|116932452|
|      B2|   Florida| 70838424|
|      WT|   Florida| 55476912|
|      WB|California| 34409076|
|      B1|California| 24462412|
|      B2|     Texas| 17170335|
|      WT|  New York| 17127828|
|      B2|  New York|  9120600|
|      WT|     Texas|  7740642|
|      F1|California|  6319924|
|      B1|   Florida|  5860134|
|      WB|     Texas|  5396118|
|      B1|     Texas|  5253339|
|      WB|   Florida|  4164054|
|      B2|  Illinois|  2796066|
|      WT|    Nevada|  2714400|
|      B2|New Jersey|  2250531|
|      E2|California|  2025296|
|      WT|  Illinois|  2001090|
+--------+----------+---------+
only showing top 20 rows



**Query on the count of people of particular visatype based on their age**

In [50]:
query1= spark.sql('''SELECT vt.visatype,if.biryear,count(*)
     FROM immigration_fact_table if     
     JOIN visatype_table vt
     ON vt.visa_type_key = if.visa_type_key
     GROUP BY 1,2
     ORDER by 3 DESC
     ''')

In [51]:
query1.show()

+--------+-------+--------+
|visatype|biryear|count(1)|
+--------+-------+--------+
|      WT| 1986.0|   32782|
|      WT| 1987.0|   30002|
|      WT| 1985.0|   29931|
|      WT| 1988.0|   29556|
|      WT| 1984.0|   28124|
|      WT| 1989.0|   27726|
|      WT| 1983.0|   26872|
|      WT| 1982.0|   25777|
|      WT| 1990.0|   25684|
|      WT| 1966.0|   25284|
|      WT| 1981.0|   24955|
|      WT| 1976.0|   24873|
|      B2| 1982.0|   24682|
|      B2| 1983.0|   24533|
|      B2| 1981.0|   24148|
|      WT| 1980.0|   24075|
|      B2| 1984.0|   23671|
|      B2| 1985.0|   23649|
|      B2| 1980.0|   23516|
|      B2| 1986.0|   23271|
+--------+-------+--------+
only showing top 20 rows



### 4.2 Data Quality Checks
The data quality checks are done to ensure that the ETL has created the fact table  and the dimension tables with adequate amount of data.
 
#### Run Quality Checks

**Check 1**

In [52]:
# Perform quality checks here

table_dfs = {
    'immigration_fact': immigration_fact_df,
    'visa_type_dim': visatype_df,
    'calendar_dim': calendar_df,
    'usa_demographics_dim': demographics_dim_df,
    'country_dim': country_dim_df
}
for table_name, table_df in table_dfs.items():
    functions.quality_checks(table_df, table_name)

Data quality check passed for immigration_fact with 3,096,313 records.
Data quality check passed for visa_type_dim with 17 records.
Data quality check passed for calendar_dim with 30 records.
Data quality check passed for usa_demographics_dim with 2,891 records.
Data quality check passed for country_dim with 229 records.


**Check 2**

In [54]:


immigrations_calender_check1 = spark.sql("""
    SELECT  COUNT(*)
    FROM calendar_table
        WHERE   id IS NULL OR id == "" OR
                arrdate IS NULL OR arrdate == "" OR
                arrival_year IS NULL OR arrival_year == "" OR
                arrival_month IS NULL OR arrival_month == "" OR
                arrival_day IS NULL OR arrival_day == "" OR
                arrival_week IS NULL OR arrival_week == "" OR
                arrival_weekday IS NULL OR arrival_weekday == ""
""")
immigrations_calender_check1.show(1)
immigrations_calender_check1.collect()[0][0]

+--------+
|count(1)|
+--------+
|       0|
+--------+



0

In [55]:

immigrations_calender_check2 = spark.sql("""
    SELECT  COUNT(*)
    FROM calendar_table
""")
immigrations_calender_check2.show(1)
immigrations_calender_check2.collect()[0][0]

+--------+
|count(1)|
+--------+
|      30|
+--------+



30

In [58]:
results={}
if immigrations_calender_check1.collect()[0][0] > 0 & immigrations_calender_check2.collect()[0][0] < 1:
    results['immigrations_count'] = immigrations_calender_check2.collect()[0][0]
    results['immigrations'] = "Not OK"
else:
    results['immigrations_count'] = immigrations_calender_check2.collect()[0][0]
    results['immigrations'] = "OK"

print(f"RESULTS: {results}")

RESULTS: {'immigrations_count': 30, 'immigrations': 'OK'}


### Step 5: Complete Project Write Up

#### Rationale for the choice of tools and technologies for the project
 
 Apache Spark was the main tool used because it is able to handle files of different format and of large amounts of data.A faster unified analytics engine to handle big data

#### Propose how often the data should be updated and why.
 
 The available I94 immigration dataset is being updated on a monthly basis . Thus our data also require monthly update.

#### Write a description of how you would approach the problem differently under the following scenarios:
 1. The data was increased by 100x.
     - Spark has the advantage of being highly scalable. So if the data is increased, we can handle it by increasing the number of our cluser nodes       
 2. The data populates a dashboard that must be updated on a daily basis by 7am every day.
     -  In this kind of a situation,we can use  Apache Airflow to schedule and run data pipelines.
 3. The database needed to be accessed by 100+ people.
     -   Amazon Redshift can be made use of in such a scenario.The analytics database can be shifted to Amazon Redshift
 .